*   Герен София
*   Горшенина Майя
*   Дамашова Заряна
*   Черноусенко Софья

# Описание проекта



**Корпус русских народных сказок**

Корпус состоит из 120 русских народных сказок. Сказки взяты с сайта https://skazki-pesni.ru.

github: https://github.com/ssofiyk/nlp_project

презентация: https://docs.google.com/presentation/d/1wqGk9YKCIW9GFz12_5Y8YYw1XM_xNIL0gUSGdl6oiFA/edit?usp=sharing

### Скачиваем и импортируем все необходимые пакеты и библиотеки соответственно


In [ ]:
!pip install pyyaml ua-parser user-agents
!pip install fake-useragent
!pip install razdel
!pip install natasha slovnet navec -q
!pip install pymorphy2

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 66.4 MB/s eta 0:00:00


In [ ]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from tqdm.auto import tqdm
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
import itertools
import re
import pandas as pd
import re
import csv
from natasha import (
    Segmenter,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    Doc
)
from pymorphy2 import MorphAnalyzer
from collections import Counter

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# 1. Парсинг

In [ ]:
response = requests.get("https://skazki-pesni.ru/tema/russkie-narodnye-skazki/")

In [ ]:
response.status_code

200

In [ ]:
ua = UserAgent()

In [ ]:
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
session = requests.session()

In [ ]:
response = session.get('https://skazki-pesni.ru/tema/russkie-narodnye-skazki/')

In [ ]:
ftales = soup.find_all('a', {'class': 'card'})

In [ ]:
# функция, обрабатывающая одну страницу сайта

def get_nth_page(page_number):
    # принимает - номер страницы
    # возвращает - три списка (тексты, заголовки, ссылки всех сказок со страницы)

    url = f'https://skazki-pesni.ru/tema/russkie-narodnye-skazki/page/{page_number}'
    req = session.get(url)
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')

    ftales = soup.find_all('a', {'class': 'card'})

    titles = []
    hrefs = []
    all_texts = []

    for tale in ftales:
        title = (tale.find('span', {'class': 'name'})).text
        href = tale.attrs['href']

        titles.append(title)
        hrefs.append(href)

    for href in hrefs:


        req = session.get(href)
        page = req.text
        soup = BeautifulSoup(page, 'html.parser')

        f_text = soup.find_all
        full_text = soup.find('div', {'class' : 'text'}).text
        full_text = full_text.replace('\n', ' ')
        all_texts.append(full_text)

    return(all_texts, titles, hrefs)

In [ ]:
# функция для сбора данных с n страниц сайта

def run_all(n_pages):
    # принимает - количество страниц
    # возвращает - список сказок с этого количества страниц
    tales = []
    for i in tqdm(range(n_pages)):
        tales.extend(get_nth_page(i+1))

    return tales

In [ ]:
all_fairy_tales = run_all(2)

  0%|          | 0/2 [00:00<?, ?it/s]

# 2. Создание базы данных

In [ ]:
df_page_1 = pd.DataFrame({
   'name': all_fairy_tales[1],
   'text': all_fairy_tales[0],
    'href': all_fairy_tales[2],
})

In [ ]:
df_page_2 = pd.DataFrame({
   'name': all_fairy_tales[4],
   'text': all_fairy_tales[3],
    'href': all_fairy_tales[5]
})

In [ ]:
df_pages = pd.concat([df_page_1, df_page_2], axis=0, sort=False)

In [ ]:
df_pages.index = [i for i in range (len(df_pages['name']))]

In [ ]:
# разбиваем на предложения

sents = []
for text in df_pages['text']:
    pattern = r'…(?=(?:\s[A-ZА-ЯЁ])|(?=\s?—\s))'
    replacement = '….'
    text = re.sub(pattern, replacement, text)
    text = re.sub(r'\s+', ' ', text)
    sentences = sent_tokenize(text)
    sents.append(sentences)

In [ ]:
# делаем датафрейм с предложениями, названием сказки, ссылкой на источник

names = list(df_pages['name'])
hrefs = list(df_pages['href'])

new_data = []
new_names = []
new_hrefs = []

for text, name, href in itertools.zip_longest(sents, names, hrefs):
    for sent in text:
        new_data.append(sent)
        new_names.append(name)
        new_hrefs.append(href)


df_sents = pd.DataFrame({'sents': new_data, 'name': new_names, 'href': new_hrefs})



In [ ]:
# добавляем предложения без пунктуации и приводим к нижнему регистру

new_sent = []
for sent in df_sents['sents']:
    sent = re.sub(r'[^\w\s-]', ' ', sent)
    sent = re.sub(r'\s+', ' ', sent)
    new_sent.append(sent.lower())

In [ ]:
df_sents['sent_without'] = new_sent

In [ ]:
df_sents.head()

,sents,name,href,sent_without
0,Cтоит в поле теремок.,Теремок,https://skazki-pesni.ru/teremok/,cтоит в поле теремок
1,Бежит мимо мышка-норушка.,Теремок,https://skazki-pesni.ru/teremok/,бежит мимо мышка-норушка
2,"Увидела теремок, остановилась и спрашивает: – ...",Теремок,https://skazki-pesni.ru/teremok/,увидела теремок остановилась и спрашивает тере...
3,Кто в тереме живёт?,Теремок,https://skazki-pesni.ru/teremok/,кто в тереме живёт
4,Никто не отзывается.,Теремок,https://skazki-pesni.ru/teremok/,никто не отзывается


In [ ]:
df_sents.to_csv("fairy_tales_razdel_full.csv", encoding='utf-8')

https://docs.google.com/spreadsheets/d/1hHpGsoV7rZtBskfFoWTIQeVx7Y-rpcezmabFvMT3qK0/edit?usp=sharing

# 3. Обработка запроса и поиск предложений

## 3.1. Лемматизация и токенизация

In [ ]:
# Инициализация модулей Natasha
segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)

# Инициализация Pymorphy2 для морфологического анализа
morphology = MorphAnalyzer()

In [ ]:
# Чтение исходного CSV файла
df = pd.read_csv('fairy_tales_razdel_full.csv')

формат файла **corpus.csv** (sep=',')


*   sents - предложение в исходном виде
*   tokens - все токены (sep=';')
*   pos_tags - части речи всех токенов (sep=';')
*   lemmas - леммы всех токенов (sep=';')
*   name - название произведения
*   href - ссылка на произведение
*   sent_without - предложение без знаков препинания и заглавных букв (через пробелы)

пример

*   – Иди ко мне жить!
*   Иди;ко;мне;жить
*   VERB;ADP;PRON;VERB
*   идти;к;я;жить
*   Теремок
*   https://skazki-pesni.ru/teremok/
*   иди ко мне жить



In [ ]:
# Открытие нового CSV файла для записи результатов
with open('corpus.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['sents', 'tokens', 'pos_tags', 'lemmas', 'name', 'href', 'sent_without'])

    # Обработка строк из исходного DataFrame
    for index, row in df.iterrows():
        sentence = row['sents']
        name = row['name']
        href = row['href']
        sent_without = row['sent_without']

        # Выполняем POS-теггинг и лемматизацию
        doc = Doc(sentence)
        doc.segment(segmenter)
        doc.tag_morph(morph_tagger)

        tokens_nat = []

        for sent in doc.sents:
            for token in sent.tokens:
                word = token.text
                tag = token.pos
                if tag != 'PUNCT':
                    lemma = morphology.parse(word)[0].normal_form  # Лемматизация через Pymorphy2
                    tokens_nat.append((word, tag, lemma))

        # Преобразуем токены для записи в CSV
        tokens_str = ";".join([t[0] for t in tokens_nat])
        pos_tags_str = ";".join([t[1] for t in tokens_nat])
        lemmas_str = ";".join([t[2] for t in tokens_nat])

        # Запись данных в CSV
        writer.writerow([sentence, tokens_str, pos_tags_str, lemmas_str, name, href, sent_without])

print("POS-теггинг и лемматизация завершены, результаты сохранены в 'corpus.csv'")

POS-теггинг и лемматизация завершены, результаты сохранены в 'corpus.csv'


## 3.2. Проверка конкретного слова на соответствие запросу

Функция для проверки всех форм слова, игнорируя регистр

In [ ]:
def check_word_form(request_part, row, word_ix):
    # принимает:
    # часть запроса (например, знать),
    # строку df с данными одного предложения
    # номер токена в этой строке, который мы проверяем

    # возвращает: bool, подходит ли этот токен под запрос

    # Лемматизируем запрос
    request_lemma = morphology.parse(request_part.lower())[0].normal_form  # Приводим запрос к нижнему регистру

    word_lemma = row['lemmas'].split(';')[word_ix]

    return request_lemma == word_lemma

Функция для проверки точной формы слова без учета регистра

In [ ]:
def check_exact_form(request_part, row, word_ix):
    # принимает:
    # часть запроса уже без кавычек (например, знает),
    # строку df с данными одного предложения
    # номер токена в этой строке, который мы проверяем

    # возвращает: bool, подходит ли этот токен под запрос

    return request_part.lower() == row['tokens'].split(';')[word_ix].lower()

Функция для проверки по лемме и тегу части речи

In [ ]:
def check_lemma_and_pos(request_part, row, word_ix):
    # принимает:
    # часть запроса (например, знать+VERB),
    # строку df с данными одного предложения
    # номер токена в этой строке, который мы проверяем

    # возвращает: bool, подходит ли этот токен под запрос

    request = request_part.split('+')
    request_lemma = request[0].lower()
    request_pos = request[1]

    word_lemma = row['lemmas'].split(';')[word_ix]
    word_pos = row['pos_tags'].split(';')[word_ix]

    if request_lemma == word_lemma and request_pos == word_pos:
        return True
    return False

Функция для проверки соответствия частеречного тега

In [ ]:
def check_pos(request_part, row, word_ix):
    # принимает:
    # часть запроса (например, VERB),
    # строку df с данными одного предложения
    # номер токена в этой строке, который мы проверяем

    # возвращает: bool, подходит ли этот токен под запрос

    word_pos = row['pos_tags'].split(';')[word_ix]
    return request_part == word_pos

Функция для вызова нужной из четырех функций выше в зависимости от запроса

In [ ]:
def word_fits_request_part(request_part, row, word_ix):
    # принимает:
    # часть запроса (например, знать+VERB),
    # строку df с данными одного предложения
    # номер токена в этой строке, который мы проверяем

    # возвращает: bool, подходит ли этот токен под запрос

    if '"' in request_part:
        # Запрос точной формы без учетам регистра
        request = request_part.strip('"').lower()
        word_res = check_exact_form(request, row, word_ix)
    elif '+' in request_part:
        # Запрос по лемме и части речи
        word_res = check_lemma_and_pos(request_part, row, word_ix)
    elif all((ord('A') <= ord(ch) <= ord('Z')) for ch in request_part):
        # Запрос по POS тегу - заглавные латинские буквы
        word_res = check_pos(request_part, row, word_ix)
    else:
        # Запрос всех форм слова, игнорируя регистр
        word_res = check_word_form(request_part, row, word_ix)
    return word_res

## 3.3. Основная функция поиска

In [ ]:
# Загрузка данных из файла
corpus_data = pd.read_csv("corpus.csv", delimiter=",")

формат структуры **matching_sentences**

строки хранятся в том виде, в котором их увидит пользователь
0. левый контекст примера
1. центральный контекст примера (соответствует запросу)
2. правый контекст примера
3. источник примера

пример

0. Те братья работают, а Емеля целый день лежит на печке,
1. знать ничего не
2. хочет.
3. Предложение из По щучьему велению https://skazki-pesni.ru/po-shhuchemu-veleniyu/

In [ ]:
def search(request, corpus_data):
    # принимает:
    # запрос (например, знать+VERB "ничего" "не"),
    # df с данными всех предложений

    # возвращает:
    # список данных всех предложений которые подошли под запрос (формат см выше)

    request_parts = request.split(' ')
    request_length = len(request_parts)
    matching_sentences = []

    for _, row in corpus_data.iterrows():
        tokens = row['tokens'].split(';')
        dirty_tokens = row['sents'].strip().replace('— ', '—').replace('– ', '–').replace(' !', '!').replace(',—', ', —').replace(' ,', ',').replace('е.Н', 'е. Н').split(' ')  # для вывода со знаками препинания

        for token_i in range(len(tokens) - request_length + 1):
            phrase_res = True  # Предполагаем, что н-грамма подойдёт
            for j in range(request_length):
                if not word_fits_request_part(request_parts[j], row, token_i + j):
                    phrase_res = False  # Н-грамма не подходит
                    break

            if phrase_res:
                # Нашли подходящую n-грамму!
                phrase_start = token_i  # номер первого токена в этой н-грамме
                phrase_end = token_i + request_length - 1
                collocation = ' '.join(dirty_tokens[phrase_start:phrase_end + 1])  # центральный контекст
                left_cont = ' '.join(dirty_tokens[:phrase_start])  # левый контекст
                right_cont = ' '.join(dirty_tokens[phrase_end + 1:])  # правый контекст
                matching_sentences.append([left_cont, collocation, right_cont, row['name'] + ' ' + row['href']])
            # Переходим к следующему токену
        # Переходим к следующему предложению
    return matching_sentences


# 4. Поиск коллокаций

Функция для поиска и вывода частотности коллокаций с введенным запросом

In [ ]:
def get_collocations(lines):
    # принимает:
    # список данных всех предложений которые подошли под запрос (формат см выше)

    # возвращает:
    # Counter содержащий коллокации
    collocations = Counter()
    for line in lines:
        # Вытаскиваем центр, левый и правый контекст
        center = re.sub(r'[^\w\s-]', '', line[1]).lower()
        left_context = re.sub(r'[^\w\s-]', '', line[0])
        right_context = re.sub(r'[^\w\s-]', '', line[2])
        # Если левый и правый контексты не пустые, то достаем коллокации для левого/правого контекстов
        # После этого записываем их в каунтер
        if left_context != '':
            left = left_context.split()[-1].lower()
            collocation_left = left + ' ' + center
            collocations[collocation_left] += 1
        if right_context != '':
            right = right_context.split()[0].lower()
            collocation_right = center + ' ' + right
            collocations[collocation_right] += 1
            # Сохраняем в список коллокации и их частотность
    return collocations


# 5. Итоговая программа

### 5.1. Вывод найденных примеров

напишем функцию для выравнивая контекстов определенного примера

In [ ]:
def pretty_line(c_left='', c_mid='', c_right='', maxlen_l=100, maxlen_m=30, maxlen_r=100, src='?'):
    # принимает - левый, центральный и правый контексты, их максимальные длины и источник
    # возвращает - строку в который все ровненько

    # выравниваем левые контексты по правому краю
    left_part = c_left.rjust(maxlen_l)
    # выравниваем центральный контекст по левому краю
    mid_part = c_mid.ljust(maxlen_m)
    # выравниваем правые контексты по левому краю
    right_part = c_right.ljust(maxlen_r)
    return left_part + ' | ' + mid_part + ' | ' + right_part + ' || Предложение из ' + src

напишем функцию, которая выводит все найденный примеры (выровнянные)

In [ ]:
def get_all_pretty_lines(data):
    # принимает - Counter содержащий коллокации
    # возвращает - список, где каждую строку нужно просто напечатать

    # найдем максимальную длину каждого контекста, чтобы выровнять
    maxlen_left = len('Левый контекст')
    maxlen_mid = len('Центральный контекст')
    maxlen_right = len('Правый контекст')
    for line in data:
        maxlen_left = max(maxlen_left, len(line[0]))
        maxlen_mid = max(maxlen_mid, len(line[1]))
        maxlen_right = max(maxlen_right, len(line[2]))

    lines_to_print = []  # результат работы функции
    if not data:
        return lines_to_print
    # внесем шапку
    lines_to_print.append(
        'Левый контекст'.ljust(maxlen_left) + ' | ' + 'Центральный контекст'.ljust(maxlen_mid) + ' | ' +
        'Правый контекст'.ljust(maxlen_right) + ' || ' + 'Источник')
    # внесем примеры
    for line in data:
        lines_to_print.append(pretty_line(line[0], line[1], line[2], maxlen_left, maxlen_mid, maxlen_right, line[3]))
    return lines_to_print

### 5.2. Вывод коллокаций

In [ ]:
def get_pretty_collocations(data):
    # принимает - список, где каждый элемент -
    # список формата [Левый_контекст, Центральный_контекст, Правый_Контекст, Источник]
    # возвращает - список, где каждую строку нужно просто напечатать

    print_collocations = []  # результат работы функции
    if not data:
        return print_collocations
    maxlen_collocation = len('Коллокация')
    maxlen_frequency = len('Число вхождений')
    for line in data.most_common():
        maxlen_collocation = max(maxlen_collocation, len(line[0]))
        maxlen_frequency = max(maxlen_frequency, len(str(line[1])))
        # внесем шапку
    print_collocations.append(
        'Коллокация'.ljust(maxlen_collocation) + ' | ' + 'Число вхождений'.ljust(maxlen_frequency))
    # внесем примеры
    for line in data.most_common():
        print_collocations.append(line[0].ljust(maxlen_collocation) + ' | ' + str(line[1]).ljust(maxlen_frequency))
    return print_collocations


###5.3. Взаимодействие с программой:

Вывод получается очень широкий, пожалуйста, пролистайте его вправо, чтобы увидеть центральные контексты

In [ ]:
# Запрос от пользователя и вывод результата
print('Введите, пожалуйста, ваш запрос:')
print()
Q = input()
examples = search(Q, corpus_data)
if not examples:
    print('По вашему запросу ничего не найдено')
else:
    print('Вот какие примеры мы нашли по вашему запросу:')
    for line in get_all_pretty_lines(examples):
        print(line)
    print(' ')
    print('А еще у нас получились вот такие коллокации с вот такими частотностями:')
    collocation_frequency = get_collocations(examples)
    pretty_collocations = get_pretty_collocations(collocation_frequency)
    for collocation in pretty_collocations:
      print(collocation)

Введите, пожалуйста, ваш запрос:

знать
Вот какие примеры мы нашли по вашему запросу:
Левый контекст                                                                                                                                                                                                                          | Центральный контекст | Правый контекст                                                                                                                                                                                                                                                                                                                                                                                                                                      || Источник
                                                                                                                                                                                                                